<h3>Chemotaxis assay image analysis</h3>

This image-analysis pipeline was used to analyze the recorded videos from the chemotaxis assays. This script was developped within the framework of the following publication:

<em>de Schaetzen & Fan et al. Random encounters and predator locomotion drive the predation of Listeria monocytogenes by Acanthamoeba castellanii.</em>

<h4>Importing the required libraries and tools</h4>

In [ ]:
from __future__ import division, unicode_literals, print_function  #for compatibility with Python 2 and 3
import os
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
import numpy as np
import pandas as pd
import trackpy as tp
import trackpy.predict
from tifffile import imsave, imread

<h4>Creating a folder structure for file saving</h4>


In [ ]:
def_path = r'F:\Chemotaxis assay analysis\PAS_Rep3_13012021'#Set the default folder, the folder mentioned here only serves as an example
#the following definition allows for easy folder creation:
def createFolder(directory):    
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
## TIFF, in this folder the converted video files will be saved (the conversion happens in ImageJ/Fiji)
TIFF_path = def_path + "\TIFF"
createFolder(TIFF_path)
## TIFFcor, in this folder the median background corrected and guassian blurred video files will be saved
TIFFcor_path = def_path + "\TIFFcor"
createFolder(TIFFcor_path)
## MED, in this folder the median background file will be saved
MED_path = def_path + "\MED"
createFolder(MED_path)
## TIFFthres, in this folder the thresholded video files will be saved
TIFFthres_path = def_path + "\TIFFthres"
createFolder(TIFFthres_path)
## DATA_FEAT: folder in which feature CSV files will be saved 
DAFE_path = def_path + "\DATA_FEAT"
createFolder(DAFE_path)
# DATA_TRAJ: folder in which trajectory CSV files will be saved  
DATR_path = def_path + "\DATA_TRAJ"
createFolder(DATR_path)  
## DATA_TRAJfilter: folder in which filtered trajectory CSV files will be saved  
DATR_f_path = def_path + "\DATA_TRAJ_filter"
createFolder(DATR_f_path)    
## DATA_TRAJ_MSD: location in which mean-squared-displacement CSV files will be saved  
DATR_MSD_path = def_path + "\DATA_TRAJ_MSD"
createFolder(DATR_MSD_path)   
## DATR_MSD_KT: location in which MSD files including translational component are saved
DATR_MSD_TC_path = def_path + "\DATA_TRAJ_MSD_TC"
createFolder(DATR_MSD_TC_path) 
## DATA_TRAJ_MSD_filter: location in which the MSD filtered trajectory CSV files will be saved  
DATR_MSD_f_path = def_path + "\DATA_TRAJ_MSD_filter"
createFolder(DATR_MSD_f_path)
## DATA_TRAJ_meanpos: location in which mean position trajectory CSV files will be saved  
DATR_meanpos_path = def_path + "\DATA_TRAJ_meanpos"
createFolder(DATR_meanpos_path)      
## DATA_BINS_prob : location in which bin analyzed CSV files will be saved
BINS_prob_path = def_path + "\DATA_BINS_prob"
createFolder(BINS_prob_path) 
## FIG_TRAJ: location in which trajectory plots will be saved  
FIGtraj_path = def_path + "\FIG_TRAJ"
createFolder(FIGtraj_path) 
## FIG_MSD_traj: location in which msd filtered trajectory plots will be saved  
FIG_MSD_traj_path = def_path + "\FIG_TRAJ_MSD_filter"
createFolder(FIG_MSD_traj_path)
## FIGmsd_path: location in which the MSD plots will be saved
FIGmsd_path = def_path + "\FIG_MSD"
createFolder(FIGmsd_path)   
## FIG_TC: location in which the translational component histograms will be saved  
FIGtc_path = def_path + "\FIG_TC"
createFolder(FIGtc_path)  

<h4>Slicing the recorded video to the required ROI</h4>

This step requires you to find the exact pixel-coordinates and width/height using the rectangle selection tool. The values for these parameters can be found in the bottom part of the ImageJ/Fiji menu-bar. Furthermore, it is assumed that the video-files have been converted from 16-bit .ND2 to 16-bit .TIFF. This can be done using the batch convert tool within ImageJ/Fiji (Process > Batch > Convert). Make sure to place the converted files into the TIFF folder.

- RAW video files were named with their timestap, but always with 2 digits (e.g. for the recording in minute 1 the file would have had the name 01.nd2). This naming protocol is used throughout the script to assign timelabels in different resulting tables.

In [ ]:
os.chdir(TIFF_path) # set the working directory to the TIFF folder
fl_TIFF = os.listdir(TIFF_path) # make a list of all filenames
i = 1; # step variable
js = 234 #Starting row for slice in pixels, in imageJ the y coordinate
ks = 300 #Starting collumn for slice in pixels
wh = 1491 #width and height of the area to be sliced in pixels
for Value in fl_TIFF: # for loop that will cycle through each video-file
    frames = np.array(imread(Value)) #import the video-file as a numpy array
    frames_slice = frames[:,js:js+wh,ks:ks+wh] #slice the file to the specified coordinates
    filename_slice = Value.replace('.tif','_SLICE.tif') #create a new filename
    imsave(filename_slice, frames_slice) #save the sliced file 
    os.remove(Value) #remove the old non-sliced file 
    print(i, 'out of', len(fl_TIFF), ' files have been sliced')
    i += 1;

<h4>Median background substraction and Gaussian blur</h4>

In [ ]:
os.chdir(TIFF_path) # set the working directory to the TIFF folder 
fl_TIFF = os.listdir(TIFF_path) # make a list of all filenames
i = 1 # step variable
R_mid = 125 # the center 8-bit value around which all pixel values will be centered
R_min = 0 # the minimum value of an 8-bit range
R_max = 255 # the maximum value of an 8-bit range
for Value in fl_TIFF: # for loop that will cycle through each video-file
    os.chdir(TIFF_path) # set the working directory to the TIFF folder 
    frames = np.array(imread(Value)) # import the video-file as a numpy array
    med_frames = np.median(frames, axis=0) # calculate the median value for each pixel over all frames
    os.chdir(MED_path) # set the working directory to the MED folder 
    filename_med = Value.replace('.tif', '_MED.tif') # create a new filename
    imsave(filename_med, med_frames) # save median pixel values
    cor_frames = np.array((frames - med_frames) + R_mid, dtype = np.int32) # 1. substract the median pixel values for each respective pixel in each frame, this centers all pixel values around 0 - 2. add the R_zero value
    cor_frames = np.where(cor_frames < R_min, R_min, cor_frames) # all pixel values lower than R_min will be set to R_min
    cor_frames = np.where(cor_frames > R_max, R_max, cor_frames) # all pixel values higher than R_max will be set to R_max
    cor_frames = np.uint8(cor_frames) # convert the corrected frames from 32-bit to 8 bit
    gblur_frames = gaussian_filter(cor_frames, sigma = (0,2,2)) # Gaussian blur with a 2x2 kernel
    os.chdir(TIFFcor_path) # set the working directory to the TIFFcor folder 
    filename_cor = Value.replace('.tif', '_COR_SCALE_GBLUR.tif') # create a new filename
    imsave(filename_cor, gblur_frames) # save the median background corrected and Gaussian blurred frames
    print(i, 'out of', len(fl_TIFF), ' .TIFF files have been median background corrected and rescaled')
    i += 1;  

<h4>Thresholding</h4>

This step requires you to find the thresholding value TV using ImageJ/Fiji (Image > Adjust > Threshold), for our experiments the TV-value was between 115-119. This has to be done for only one video-file for one experiment, assuming that no experimental setup changes where made during the microscopic image acquisition.

In [ ]:
os.chdir(TIFFcor_path) # set the working directory to the TIFFcor folder
fl_TIFF = os.listdir(TIFFcor_path) # make a list of all filenames
TV = 116 # This value was determined using the threshold function in ImageJ, this value should be between 115-119
i = 1;
for Value in fl_TIFF: # for loop that will cycle through each video-file
    os.chdir(TIFFcor_path) # set the working directory to the TIFFcor folder
    frames = np.array(imread(Value)) # import the video-file as a numpy array
    thres_frames = np.where(frames <= TV, frames, 255) # replace all pixel values higher than the TV-value with 255
    filename_thres = Value.replace('.tif', '_THRES.tif') # create a new filename
    os.chdir(TIFFthres_path) # set the working directory to the TIFFthres folder
    imsave(filename_thres, thres_frames) # save the thresholded frames
    print(i, 'out of', len(fl_TIFF), ' files have been gaussian blurred and thresholded')
    i += 1;

<h4>Particle recognition and trajectory formation</h4>

For more information regarding the TrackPy, we refer to the in-depth information provided by the TrackPy team on their github: http://soft-matter.github.io/trackpy/v0.5.0/. Furthermore, prior to the next analyses steps, the TrackPy parameters should be determined using the ACLI_TrackPy_parameterization script which can be found in the following github repository: https://github.com/Renderfarm/Acanthamoeba-Listeria.

In [ ]:
os.chdir(TIFFthres_path) # set the working directory to the TIFFthres folder
fl_THRES = os.listdir(TIFFthres_path) # make a list of all filenames
i = 1; # step variable
Dia = 9 # diameter: approximate diameter (in pixels) of the particles to be located
MinMass = 0 # Minimum mass: the minimum integrated brightness/darkness (filter to remove spurious objects)
Nsize = 1 # Noise size: width of the gaussian blur filter to remove noise
sep = 5 # minumum separation (in pixels) between two features
Pmem = 15 # Particle memory: the maximum number of frames during which a feature can vanish, then reappear nearby, and be considered the same particle.
TrackTresh = 5 # Minimum number of points (video frames) to survive
MaxT = 5 # Maximum amount of pixels a bacteria can move in between two consecutive frames
for Value in fl_THRES: # for loop that will cycle through each video-file
    os.chdir(TIFFthres_path) # set the working directory to the TIFFthres folder
    frames = np.array(imread(Value) # import the frames as a numpy array
    ## Batch locate particles in each frame of the stack with tp.batch(array, diameter, invert = True, noise_size, minmass, separation)
    fe = tp.batch(frames, diameter = Dia, invert = True, noise_size = Nsize, minmass = MinMass, separation = sep);
    os.chdir(DAFE_path) # set the working directory to the DAFE folder
    filename_fe = Value.replace('.tif', '_feat.csv') # create a new filename
    pd.DataFrame(fe).to_csv(filename_fe) # save the feature dataframe to a .csv file
    ## predictive trajectory linking
    pred = trackpy.predict.NearestVelocityPredict() #activate the predictive tracking feature of TrackPy
    tr = pred.link_df(fe, search_range = MaxT, memory = Pmem) # create trajectories with the predictive tracking feature 
    os.chdir(DATR_path) # set the working directory to the DATR folder
    filename_tr = Value.replace('.tif', '_traj.csv') # create a new filename
    pd.DataFrame(tr).to_csv(filename_tr)# save the trajectory dataframe to a .csv file
    ## Filter trajectories based on their occurance in consecutive frames
    trf = tp.filter_stubs(tr, TrackTresh) # filter out trajectoreies based on TrackTresh
    filename_trf = Value.replace('.tif', '_trajfilter.csv') # create a new filename
    os.chdir(DATR_f_path)# set the working directory to the DATR_f folder
    pd.DataFrame(trf).to_csv(filename_trf) # save the filtered trajectory dataframe to a .csv file
    ## Plot of trajectories colored by particle
    plt.figure(figsize = (5,5)) # create an empty plot
    ax1 = plt.subplot(1,1,1) # add axes to the plot
    tp.plot_traj(trf, ax=ax1,colorby = 'particle') # plot the trajectories
    plotname_trf = Value.replace('.tif', '_traj.tif') # create a new plotname
    os.chdir(FIGtraj_path) # set the working directory to the FIGtraj_path folder
    plt.savefig(plotname_trf, format = 'tif', dpi = 300)  # save the plots as a tif file
    print(i, 'out of', len(fl_TIFF), ' files have been analyzed ofr trajectories')
    i += 1;

<h4>Mean squared displacement (MSD) + translational component calculations</h4>

In [ ]:
os.chdir(DATR_f_path) # set the working directory to the DATR_f folder
fl_DATA = os.listdir(DATR_f_path) # create a list of all filenames
# input parameters for MSD calculations
x_lim = 1491 # width of the images in pixel value, determined in the slicing step
w_ch = 1000 # width of the channel in micrometers
fr_ps = 25 # fps of recorded videos
l_t = 25 # maximum lag time
tot_fr = 125 # total amount of frames in videos
i = 1 # step variable
for Value in fl_DATA: # for loop that will cycle through each .csv-file
    ## calculate the MSD's for each trajectory
    os.chdir(DATR_f_path) # set the working directory to the DATR_f folder
    trf = pd.read_csv(Value) # import the .csv file as a DataFrame
    msd = tp.imsd(trf, w_ch/x_lim, fps = fr_ps, max_lagtime = l_t) # calculate the MSD of each trajectory
    os.chdir(DATR_MSD_path) # set the working directory to the DATR_MSD folder
    filename_msd = Value.replace('.csv', '_msd.csv') # create a new filename
    pd.DataFrame(msd).to_csv(filename_msd) # save the MSD DataFrame as a .csv file
    ## plot figure off all MSD's
    plt.figure(figsize = (5,5)) # create an empty plot
    ax_msd = plt.subplot(1,1,1) # add axes to the plot
    ax_msd.plot(msd.index, msd, 'k-', alpha=0.1) # plot the MSD's
    ax_msd.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]', xlabel='lag time $t$') # label the axes
    ax_msd.set_xscale('log') # rescale the x-axis to logarithmic
    ax_msd.set_yscale('log') # rescale the y-axis to logarithmic
    plotname_msd = Value.replace('.csv', '_MSD.tif') # create a new plotname
    os.chdir(FIGmsd_path) # set the working directory to the FIGmsd folder
    plt.savefig(plotname_msd, format = 'tif', dpi = 600) # save the MSD plot 
    ## calculate the translational component for filtering purposes
    msdt = msd.transpose() # transposed the DataFrame to calculate the translational component
    msdt.drop(msdt.loc[msdt[1.0] == 0].index, inplace=True)
    msdt['tc'] = ((np.log10(msdt[l_t/fr_ps]) - np.log10(msdt[1/fr_ps]))/(np.log10(l_t/fr_ps)-np.log10(1/fr_ps))) # calculate the translational component based on the MSD curves
    msdt = msdt.dropna() # drops rows from the dataframe that have no values
    os.chdir(DATR_MSD_TC_path) # set the working directory to the DATR_MSD_TC folder
    filename_tc = Value.replace('.csv', '_tc.csv') # create a new filename
    pd.DataFrame(msdt).to_csv(filename_tc) # save the msdt DataFrame including tc data as a .csv file
    ## plot a histogram of translational component
    plt.figure(figsize = (5,5)) # create an empty plot
    ax_tc = plt.subplot(1,1,1) # add axes to the plot
    ax_tc.set_xlim([0,2]) # set the x-axis limit
    plt.xticks(np.arange(0, 2.2, 0.2)) # set the x-axis ticks
    plt.hist(msdt['tc'], bins = np.arange(0,2.1, 0.1)) # plot a histogram of tc data
    plotname = Value.replace('.csv', '_tc.tif') # create a new plotname
    os.chdir(FIGtc_path) # set the working directory to the FIG_TC folder
    plt.savefig(plotname, format = 'tif', dpi = 600) # save the tc plot 
    print(i, 'out of', len(fl_DATA), ' files have been analyzed for msd and tc')
    i += 1

<h4>Filter out trajectories that are subdiffusive based on their translational component</h4>

In [ ]:
os.chdir(DATR_MSD_TC_path) # set the working directory to the DATR_MSD_KT folder
fl_DATR_MSD_tc = os.listdir(DATR_MSD_TC_path) # create a list of filenames
co_tc = 1.2 # cutoff value for tc
x_lim = 1491 # width of the image in pixels
y_lim = 1491 # height of the image in pixels
i = 1 # step variable
for Value in fl_DATR_MSD_tc:  # for loop that will cycle through each csv-file
    ## create a list of particles with their unique ID that have a translational component larger or equal than 1.2
    os.chdir(DATR_MSD_TC_path) # set the working directory to the DATR_MSD_KT folder
    msdt_tc = pd.read_csv(Value, index_col = 0) # import the .csv file as a DataFrame
    msdt_tc = msdt_tc.loc[msdt_tc['tc'] >= co_tc] # filter out particles that are not-motile/subdiffusive
    dpart =  list(msdt_tc.index.values) # make a list of all particles that are considered to be motile
    filename_fe = Value.replace('_tc.csv', '.csv') # create a filename to call on a previously made trajectory .csv file
    os.chdir(DATR_f_path) # set the working directory to the DATR_f folder
    trf = pd.read_csv(filename_fe) # import the .csv file as a DataFrame
    ## filter out all bacteria that did not meet the cut-off value
    trf = trf.loc[trf['particle'].isin(dpart)] #filter out particles that have a tc value lower or equal than 1.2
    os.chdir(DATR_MSD_f_path) # set the working directory to the DATR_MSD_f folder
    filename_trf = filename_fe.replace('.csv', '_MSDfilter.csv') # create a new filename
    pd.DataFrame(trf).to_csv(filename_trf) # save the filtered trajectory Dataframe to a .csv file
    ## plot the filtered trajectories
    plt.figure(figsize = (5,5)) # create an empty plot
    ax1 = plt.subplot(1,1,1) # add axes to the plot
    ax1.set_xlim([0,x_lim]) # set the x-axis limit
    ax1.set_ylim([y_lim,0]) # set the y-axis limit
    tp.plot_traj(trf, ax=ax1,colorby = 'particle') # plot the trajectories
    plotname = Value.replace('.csv', '_MSD_traj.tif') # create a new plotname
    os.chdir(FIG_MSD_traj_path) # set the working directory to the FIG_MSD_traj folder
    plt.savefig(plotname, format = 'tif', dpi = 600) # save the plot as a .tif file
    print(i, 'out of', len(fl_DATR_MSD_tc), ' files have been msd filtered for non-motile bacteria')
    i += 1    

<h4>Calculate the mean position for each bacteria per timepoint </h4>

In [ ]:
os.chdir(DATR_MSD_f_path) # set the working directory to the DATR_MSD_f folder
fl_DATR_MSD_f = os.listdir(DATR_MSD_f_path) # create a list of all filenames
i = 1 # step variable
for Value in fl_DATR_MSD_f: # for loop that will cycle through each .csv-file
    os.chdir(DATR_MSD_f_path) # set the working directory to the DATR_MSD_f folder
    trf = pd.read_csv(Value) # import the .csv file as a DataFrame
    part = pd.DataFrame(trf.particle.unique(), columns = ['pID']) # create a Dataframe with a list of unique particle ID's
    mp = pd.DataFrame() # create an empty DataFrame
    for pID in part['pID']: # for loop that will cycle through each particle ID
        pos = trf[trf.particle == pID] # for a given particle ID collect all positions
        mx = pos['x'].mean() # calculate the mean x coordinate over all frames
        my = pos['y'].mean() # calculate the mean y coordinate over all frames
        stdevx = np.std(pos['x']) # calculate the stdev of x coordinates
        stdevy = np.std(pos['y']) # calculate the stdev of y coordinates
        tframes = (pos.frame.max() + 1) - (pos.frame.min() + 1) # calculate the amount of frames a particle was tracked (the +1 modifier is to account for the starting frame label of 0) 
        data = [(pID, mx, my, stdevx, stdevy, tframes)] # combine all previously calculated variables in a list
        mps = pd.DataFrame(data, columns = ['pID','mx','my','stdevx','stdevy','tframes']) # add all calculated data to a DataFrame
        mp = mp.append(mps, sort = False) # merge DataFrames into one 
    filename_mp = Value.replace('.csv', '_mp.csv') # create a new filename
    os.chdir(DATR_meanpos_path) # set the working directory to the DATR_meanpos folder 
    pd.DataFrame(mp).to_csv(filename_mp) # save the mean position DataFrame to a .csv file
    print(i, 'out of', len(fl_DATR_MSD_f), ' files have been analyzed for the bacterial mean position')
    i += 1

<h4>Calculate the relative cell concentrations over a number of bins for a chemotactic reponse from RIGHT to LEFT </h4>

Use this script-block only when the chemical attractant is generated from the LEFT!
Other versions of this script for chemical attractants generated from another direction can be found below!

In [ ]:
bins = 5 # amount of bins
x_lim = 1500 # width of the image in pixels
w_ch = 1000 # width of the channel in microns
x_step = x_lim/bins # width of each bin in pixels
mpp = w_ch/x_lim # micron per pixel, conversion factor
os.chdir(DATR_meanpos_path) # set the working directory to the DATR_meanpos folder
fl_DATR_meanpos = os.listdir(DATR_meanpos_path) # create a list of all filenames
rel_conc = pd.DataFrame() # create an empty DataFrame
i = 1 # step variable
for Value in fl_DATR_meanpos: # for loop that will cycle through meanposition csv-file
    os.chdir(DATR_meanpos_path) # set the working directory to the DATR_meanpos folder
    mp = pd.read_csv(Value) # import the .csv file as a DataFrame
    b = 1 # step variable for bins
    x0 = 0 # starting coordinate
    bin_cc = x_step/2 # central coordinate in pixels of each bin for plotting purposes
    dbin = pd.DataFrame(columns=['x', 'count']) # create an empty DataFrame 
    while b <= bins: # a while function that will cycle through each bin
        mp_bin = mp[(mp.mx >= x0) & (mp.mx <= (x0 + x_step))] # Select all mean positions that are situated within the borders of that given bin
        counts = np.array(mp_bin['pID'].nunique()) # count the amount of unique particel ID's
        bin_count = [(bin_cc, counts)] # create a list of the counts and central bin coordinate
        dbin_row = pd.DataFrame(bin_count, columns=['x', 'count']) # make a Dataframe row of the 
        dbin = dbin.append(dbin_row, sort = False) # append the row to the bin data
        x0 += x_step # increase the starting coordinate with the binwidth
        bin_cc += x_step # increase the central bin coordinate with the binwidth
        b += 1 # increase the bin step variable
    dbin['rel_conc'] = dbin['count']/dbin['count'].sum() # calculate the relative concentration for each bin
    dbin['x_um'] = dbin['x']*mpp # calculate the central coordinate of each bin in micrometers
    os.chdir(BINS_prob_path)# set the working directory to the BINS_prob_path
    filename_prob = Value.replace('.csv', '_bins.csv') # create a new filename
    pd.DataFrame(dbin).to_csv(filename_prob) # save the the relative concentrations per bin for each individual timepoint in a .csv file
    values = [(os.path.splitext(Value[0:2])[0], dbin.iloc[0]['rel_conc'], dbin.iloc[1]['rel_conc'], dbin.iloc[2]['rel_conc'], dbin.iloc[3]['rel_conc'], dbin.iloc[4]['rel_conc'])] # take the relative concentrations from each bin
    rel_conc_row = pd.DataFrame(values, columns = ['time', 'R1', 'R2', 'R3', 'R4', 'R5']) # put all values in a DataFrame row
    rel_conc = rel_conc.append(rel_conc_row, sort = False) # combine all rows into one DataFrame
    print(i, 'out of', len(fl_DATR_meanpos), ' files have been analyzed for their relative concentrations in each bin')
    i += 1   
os.chdir(def_path) # set the working directory to the default folder location
pd.DataFrame(rel_conc).to_csv('Relative_concentration_bins.csv') # save the the relative concentrations per bin for all timepoints in a .csv file

<h4>Calculate the relative cell concentrations over a number of bins for a chemotactic reponse from LEFT to RIGHT </h4>

Use this script-block only when the chemical attractant is generated from the RIGHT!
The only difference between this block and the previous is the order in which the relative concentrations are collected and saved into the Relative_concentration_bins.csv.

In [ ]:
bins = 5 # amount of bins
x_lim = 1975 # width of the image in pixels
w_ch = 1000 # width of the channel in microns
x_step = x_lim/bins # width of each bin in pixels
mpp = w_ch/x_lim # micron per pixel, conversion factor
os.chdir(DATR_meanpos_path) # set the working directory to the DATR_meanpos folder
fl_DATR_meanpos = os.listdir(DATR_meanpos_path) # create a list of all filenames
rel_conc = pd.DataFrame() # create an empty DataFrame
i = 1 # step variable
for Value in fl_DATR_meanpos: # for loop that will cycle through meanposition csv-file
    os.chdir(DATR_meanpos_path) # set the working directory to the DATR_meanpos folder
    mp = pd.read_csv(Value) # import the .csv file as a DataFrame
    b = 1 # step variable for bins
    x0 = 0 # starting coordinate
    bin_cc = x_step/2 # central coordinate in pixels of each bin for plotting purposes
    dbin = pd.DataFrame(columns=['x', 'count']) # create an empty DataFrame 
    while b <= bins: # a while function that will cycle through each bin
        mp_bin = mp[(mp.mx >= x0) & (mp.mx <= (x0 + x_step))] # Select all mean positions that are situated within the borders of that given bin
        counts = np.array(mp_bin['pID'].nunique()) # count the amount of unique particel ID's
        bin_count = [(bin_cc, counts)] # create a list of the counts and central bin coordinate
        dbin_row = pd.DataFrame(bin_count, columns=['x', 'count']) # make a Dataframe row of the 
        dbin = dbin.append(dbin_row, sort = False) # append the row to the bin data
        x0 += x_step # increase the starting coordinate with the binwidth
        bin_cc += x_step # increase the central bin coordinate with the binwidth
        b += 1 # increase the bin step variable
    dbin['rel_conc'] = dbin['count']/dbin['count'].sum() # calculate the relative concentration for each bin
    dbin['x_um'] = dbin['x']*mpp # calculate the central coordinate of each bin in micrometers
    os.chdir(BINS_prob_path)# set the working directory to the BINS_prob_path
    filename_prob = Value.replace('.csv', '_bins.csv') # create a new filename
    pd.DataFrame(dbin).to_csv(filename_prob) # save the the realtiove concentrations per bin for each timepoint in a .csv file
    values = [(os.path.splitext(Value[0:2])[0], dbin.iloc[4]['rel_conc'], dbin.iloc[3]['rel_conc'], dbin.iloc[2]['rel_conc'], dbin.iloc[1]['rel_conc'], dbin.iloc[0]['rel_conc'])] # take the relative concentrations from each bin
    rel_conc_row = pd.DataFrame(values, columns = ['time', 'R1', 'R2', 'R3', 'R4', 'R5']) # put all values in a DataFrame row
    rel_conc = rel_conc.append(rel_conc_row, sort = False) # combine all rows into one DataFrame
    print(i, 'out of', len(fl_DATR_meanpos), ' files have been analyzed for their relative concentrations in each bin')
    i += 1   
os.chdir(def_path) # set the working directory to the default folder location
pd.DataFrame(rel_conc).to_csv('Relative_concentration_bins.csv') # save the the relative concentrations per bin for all timepoints in a .csv file    

<h4>Calculate the relative cell concentrations over a number of bins for a chemotactic reponse from TOP to BOTTOM </h4>

Use this script-block only when the chemical attractant is generated from the BOTTOM!

In [ ]:
bins = 5 # amount of bins
y_lim = 1491 # width of the image in pixels
w_ch = 1000 # width of the channel in microns
y_step = y_lim/bins # width of each bin in pixels
mpp = w_ch/y_lim # micron per pixel, conversion factor
os.chdir(DATR_meanpos_path) # set the working directory to the DATR_meanpos folder
fl_DATR_meanpos = os.listdir(DATR_meanpos_path) # create a list of all filenames
rel_conc = pd.DataFrame() # create an empty DataFrame
i = 1 # step variable
for Value in fl_DATR_meanpos: # for loop that will cycle through meanposition csv-file
    os.chdir(DATR_meanpos_path) # set the working directory to the DATR_meanpos folder
    mp = pd.read_csv(Value) # import the .csv file as a DataFrame
    b = 1 # step variable for bins
    y0 = 0 # starting coordinate
    bin_cc = y_step/2 # central coordinate in pixels of each bin for plotting purposes
    dbin = pd.DataFrame(columns=['y', 'count']) # create an empty DataFrame 
    while b <= bins: # a while function that will cycle through each bin
        mp_bin = mp[(mp.my >= y0) & (mp.my <= (y0 + y_step))] # Select all mean positions that are situated within the borders of that given bin
        counts = np.array(mp_bin['pID'].nunique()) # count the amount of unique particel ID's
        bin_count = [(bin_cc, counts)] # create a list of the counts and central bin coordinate
        dbin_row = pd.DataFrame(bin_count, columns=['y', 'count']) # make a Dataframe row of the 
        dbin = dbin.append(dbin_row, sort = False) # append the row to the bin data
        y0 += y_step # increase the starting coordinate with the binwidth
        bin_cc += y_step # increase the central bin coordinate with the binwidth
        b += 1 # increase the bin step variable
    dbin['rel_conc'] = dbin['count']/dbin['count'].sum() # calculate the relative concentration for each bin
    dbin['y_um'] = dbin['y']*mpp # calculate the central coordinate of each bin in micrometers
    os.chdir(BINS_prob_path)# set the working directory to the BINS_prob_path
    filename_prob = Value.replace('.csv', '_bins.csv') # create a new filename
    pd.DataFrame(dbin).to_csv(filename_prob) # save the the realtiove concentrations per bin for each timepoint in a .csv file
    values = [(os.path.splitext(Value[0:2])[0], dbin.iloc[4]['rel_conc'], dbin.iloc[3]['rel_conc'], dbin.iloc[2]['rel_conc'], dbin.iloc[1]['rel_conc'], dbin.iloc[0]['rel_conc'])] # take the relative concentrations from each bin
    rel_conc_row = pd.DataFrame(values, columns = ['time', 'R1', 'R2', 'R3', 'R4', 'R5']) # put all values in a DataFrame row
    rel_conc = rel_conc.append(rel_conc_row, sort = False) # combine all rows into one DataFrame
    print(i, 'out of', len(fl_DATR_meanpos), ' files have been analyzed for their relative concentrations in each bin')
    i += 1   
os.chdir(def_path) # set the working directory to the default folder location
pd.DataFrame(rel_conc).to_csv('Relative_concentration_bins.csv') # save the the relative concentrations per bin for all timepoints in a .csv file    

<h4>Calculate the relative cell concentrations over a number of bins for a chemotactic reponse from BOTTOM to TOP </h4>

Use this script-block only when the chemical attractant is generated from the TOP!

In [ ]:
bins = 5 # amount of bins
y_lim = 1467 # width of the image in pixels
w_ch = 1000 # width of the channel in microns
y_step = y_lim/bins # width of each bin in pixels
mpp = w_ch/y_lim # micron per pixel, conversion factor
os.chdir(DATR_meanpos_path) # set the working directory to the DATR_meanpos folder
fl_DATR_meanpos = os.listdir(DATR_meanpos_path) # create a list of all filenames
rel_conc = pd.DataFrame() # create an empty DataFrame
i = 1 # step variable
for Value in fl_DATR_meanpos: # for loop that will cycle through meanposition csv-file
    os.chdir(DATR_meanpos_path) # set the working directory to the DATR_meanpos folder
    mp = pd.read_csv(Value) # import the .csv file as a DataFrame
    b = 1 # step variable for bins
    y0 = 0 # starting coordinate
    bin_cc = y_step/2 # central coordinate in pixels of each bin for plotting purposes
    dbin = pd.DataFrame(columns=['y', 'count']) # create an empty DataFrame 
    while b <= bins: # a while function that will cycle through each bin
        mp_bin = mp[(mp.my >= y0) & (mp.my <= (y0 + y_step))] # Select all mean positions that are situated within the borders of that given bin
        counts = np.array(mp_bin['pID'].nunique()) # count the amount of unique particel ID's
        bin_count = [(bin_cc, counts)] # create a list of the counts and central bin coordinate
        dbin_row = pd.DataFrame(bin_count, columns=['y', 'count']) # make a Dataframe row of the 
        dbin = dbin.append(dbin_row, sort = False) # append the row to the bin data
        y0 += y_step # increase the starting coordinate with the binwidth
        bin_cc += y_step # increase the central bin coordinate with the binwidth
        b += 1 # increase the bin step variable
    dbin['rel_conc'] = dbin['count']/dbin['count'].sum() # calculate the relative concentration for each bin
    dbin['y_um'] = dbin['y']*mpp # calculate the central coordinate of each bin in micrometers
    os.chdir(BINS_prob_path)# set the working directory to the BINS_prob_path
    filename_prob = Value.replace('.csv', '_bins.csv') # create a new filename
    pd.DataFrame(dbin).to_csv(filename_prob) # save the the realtiove concentrations per bin for each timepoint in a .csv file
    values = [(os.path.splitext(Value[0:2])[0], dbin.iloc[0]['rel_conc'], dbin.iloc[1]['rel_conc'], dbin.iloc[2]['rel_conc'], dbin.iloc[3]['rel_conc'], dbin.iloc[4]['rel_conc'])] # take the relative concentrations from each bin
    rel_conc_row = pd.DataFrame(values, columns = ['time', 'R1', 'R2', 'R3', 'R4', 'R5']) # put all values in a DataFrame row
    rel_conc = rel_conc.append(rel_conc_row, sort = False) # combine all rows into one DataFrame
    print(i, 'out of', len(fl_DATR_meanpos), ' files have been analyzed for their relative concentrations in each bin')
    i += 1   
os.chdir(def_path) # set the working directory to the default folder location
pd.DataFrame(rel_conc).to_csv('Relative_concentration_bins.csv') # save the the relative concentrations per bin for all timepoints in a .csv file    